In [1]:
import json
from shapely.geometry import shape, Point
import datetime
from datetime import date, time
import time
import numpy as np
import pandas as pd
from src.utils import parse_coord_tuple


In [2]:
with open('./data/json_1_states.json') as f: 
    states_gj = json.load(f)  # gj = geojson

def reverse_geocode(lon,lat):
    point = Point(lon, lat) # lon/lat
    for feature in states_gj['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point): return (feature['properties'])['state'].title()
        # if polygon.contains(point): return [(feature['properties'])['ADM1_EN'], (feature['properties'])['ADM2_EN']]

    return 'None'

df = pd.read_excel("./output/postcode_output.xlsx", sheet_name="Postcodes", dtype={"postcode": str, "master_postcode": str}, converters={"lat_lon": parse_coord_tuple})

df.head()


,Unnamed: 0,postcode,master_postcode,state,ppv_region,lat_lon
0,0,76100,76100,Melaka,Melaka,"(2.26240649, 102.272489135)"
1,1,70200,70000,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
2,2,70300,70000,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
3,4,71950,70000,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
4,5,43000,43000,Selangor,Selangor/KL/Putrajaya,"(2.99319, 101.787)"


In [7]:
df["state_gj"] = df["lat_lon"].apply(lambda x: reverse_geocode(x[1], x[0]))  # reversed lat/lon!

In [19]:
df = df[["postcode", "state", "state_gj", "lat_lon"]]
display(df.state_gj.drop_duplicates())
display(df.state.drop_duplicates())

ok_states = df.state.drop_duplicates().to_list()

same_states = {
    "Negeri": "Negeri Sembilan",
    "W.P. Kuala Lumpur": "WP Kuala Lumpur",
    "W.P. Putrajaya": "WP Putrajaya",
    "W.P. Labuan": "WP Labuan"
}

df["state_gj"] = df.apply(lambda x: x["state_gj"] if x["state_gj"] in ok_states else same_states[x["state_gj"]], axis=1)
display(df)

0                  Melaka
1                  Negeri
4                Selangor
7                  Pahang
8                   Johor
22               Kelantan
30                  Perak
42      W.P. Kuala Lumpur
95         W.P. Putrajaya
134          Pulau Pinang
184               Sarawak
205            Terengganu
243                 Kedah
247                Perlis
712                 Sabah
1430          W.P. Labuan
Name: state_gj, dtype: object

0                Melaka
1       Negeri Sembilan
4              Selangor
7                Pahang
8                 Johor
22             Kelantan
30                Perak
42      WP Kuala Lumpur
95         WP Putrajaya
134        Pulau Pinang
184             Sarawak
205          Terengganu
243               Kedah
247              Perlis
712               Sabah
1430          WP Labuan
Name: state, dtype: object

,postcode,state,state_gj,lat_lon
0,76100,Melaka,Melaka,"(2.26240649, 102.272489135)"
1,70200,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
2,70300,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
3,71950,Negeri Sembilan,Negeri Sembilan,"(2.71934, 101.941512)"
4,43000,Selangor,Selangor,"(2.99319, 101.787)"
...,...,...,...,...
2881,88875,Sabah,Sabah,"(5.982556, 116.074506)"
2882,89707,Sabah,Sabah,"(5.52718611, 115.857611)"
2883,91128,Sabah,Sabah,"(5.024247, 118.330774)"
2884,32100,Perak,Perak,"(4.21121389, 100.641323)"


In [22]:
uhoh = df[df["state"] != df["state_gj"]].sort_values(["lat_lon", "postcode"])
uhoh.to_csv("./output/posm_coords_not_in_state_geojson.csv")